# Producing Mercury Challenge MA Baserate Models

This notebook illustrates a backtest of the Mercury Challenge Baserate models for Military Activity.  See [Developing Mercury Time Series Models](./Fit%20CU%20Time%20Series%20Models.ipynb) for model development details.

In [1]:
%matplotlib inline

In [2]:
import os, re
import json
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import (plot_acf, plot_pacf)
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
import datetime
from dateutil.parser import parse
import sys
sys.path.append(("../.."))
from Baserate.main.baserate import (
    MaBaserate
)
from ExpressScore.main.express_score import MaScorer
import warnings
warnings.filterwarnings("ignore")

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use("fivethirtyeight");

In [3]:
EVT_ABBR = "MA"
MC_HOME = os.path.abspath("../../..")
DOC_PATH = os.path.join(MC_HOME, "doc")
EVT_DOC_PATH = os.path.join(DOC_PATH, "scoring", EVT_ABBR)
DATA_PATH = os.path.join(MC_HOME, "data")
GSR_PATH = os.path.join(DATA_PATH, "gsr")
MA_GSR_PATH = os.path.join(GSR_PATH, "ma_gsr")
WARN_PATH = os.path.join(DATA_PATH, "baserate_warnings")
MA_WARN_PATH = os.path.join(WARN_PATH, "baserate_ma_warnings")
RESOURCE_PATH = os.path.join(MC_HOME, "src", "Baserate", "resources")
HISTORY_LOOKBACK = 365
HISTORY_DELAY = 14
WARN_START_DATE = "2018-09-19"
WARN_END_DATE = "2018-10-02"
test_range = pd.date_range(WARN_START_DATE, WARN_END_DATE)
test_months = set([dd.strftime("%B_%Y") for dd in test_range])
PARTICIPANT_ID = "Mercury_Baserate"
MA_COUNTRIES = ["Bahrain", "Egypt", "Iraq", "Jordan",
                "Lebanon", "Qatar", "Saudi Arabia", "Syria"]

In [4]:
test_months

{'October_2018', 'September_2018'}

Load the GSR from the directory

In [5]:
gsr_files = [x for x in os.listdir(MA_GSR_PATH) if x.endswith("json")]
print(gsr_files)
gsr = []
for gf in gsr_files:
    with open(os.path.join(MA_GSR_PATH, gf), "r", encoding="utf-8-sig") as f:
        monthly_gsr = json.load(f)
        gsr += monthly_gsr
print(len(gsr))

['MA_April_2016.json', 'MA_April_2017.json', 'MA_April_2018.json', 'MA_August_2015.json', 'MA_August_2016.json', 'MA_August_2017.json', 'MA_August_2018.json', 'MA_December_2015.json', 'MA_December_2016.json', 'MA_December_2017.json', 'MA_February_2016.json', 'MA_February_2017.json', 'MA_February_2018.json', 'MA_January_2016.json', 'MA_January_2017.json', 'MA_January_2018.json', 'MA_July_2015.json', 'MA_July_2016.json', 'MA_July_2017.json', 'MA_July_2018.json', 'MA_June_2015.json', 'MA_June_2016.json', 'MA_June_2017.json', 'MA_June_2018.json', 'MA_March_2016.json', 'MA_March_2017.json', 'MA_March_2018.json', 'MA_May_2015.json', 'MA_May_2016.json', 'MA_May_2017.json', 'MA_May_2018.json', 'MA_November_2015.json', 'MA_November_2016.json', 'MA_November_2017.json', 'MA_October_2015.json', 'MA_October_2016.json', 'MA_October_2017.json', 'MA_September_2015.json', 'MA_September_2016.json', 'MA_September_2017.json', 'MA_September_2018.json']
9582


In [7]:
for cc in MA_COUNTRIES:
    print(cc)
    cc_path_str = re.sub(" ", "_", cc)
    br = MaBaserate(cc)
    cc_gsr = [x for x in gsr if x["Country"] == cc]
    print("There are {} GSR events".format(len(cc_gsr)))

    try:
        preds=br.make_predictions(WARN_START_DATE, WARN_END_DATE, history_delay=HISTORY_DELAY,
                                  gsr=cc_gsr)
        print(len(preds))

        pred_json = br.convert_warnings_to_json(preds)
    except AttributeError:
        pred_json = {"participant_id": "Mercury_Baserate", "payload": []}

    out_filename = "Baserate_MA_{0}_{1}_to_{2}.json".format(cc_path_str, 
                                                            WARN_START_DATE, WARN_END_DATE)
    out_path = os.path.join(MA_WARN_PATH, cc_path_str, out_filename)
    with open(out_path, "w") as f:
        json.dump(pred_json, f, ensure_ascii=False, indent=2)
    print(out_path)

Bahrain
There are 3 GSR events
0
/Users/peterhaglich/Dropbox/Work/IARPA/Mercury/peterhaglich/mercury-challenge/data/baserate_warnings/baserate_ma_warnings/Bahrain/Baserate_MA_Bahrain_2018-09-19_to_2018-10-02.json
Egypt
There are 254 GSR events
2
/Users/peterhaglich/Dropbox/Work/IARPA/Mercury/peterhaglich/mercury-challenge/data/baserate_warnings/baserate_ma_warnings/Egypt/Baserate_MA_Egypt_2018-09-19_to_2018-10-02.json
Iraq
There are 3080 GSR events
34
/Users/peterhaglich/Dropbox/Work/IARPA/Mercury/peterhaglich/mercury-challenge/data/baserate_warnings/baserate_ma_warnings/Iraq/Baserate_MA_Iraq_2018-09-19_to_2018-10-02.json
Jordan
There are 13 GSR events
0
/Users/peterhaglich/Dropbox/Work/IARPA/Mercury/peterhaglich/mercury-challenge/data/baserate_warnings/baserate_ma_warnings/Jordan/Baserate_MA_Jordan_2018-09-19_to_2018-10-02.json
Lebanon
There are 205 GSR events
1
/Users/peterhaglich/Dropbox/Work/IARPA/Mercury/peterhaglich/mercury-challenge/data/baserate_warnings/baserate_ma_warnings/Le